In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
c.tail(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
16243,2020-05-08,Zimbabwe,0,0,34,4
16244,2020-05-09,Zimbabwe,1,0,35,4
16245,2020-05-10,Zimbabwe,1,0,36,4


In [3]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [4]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-09,France,139103.0,0.0,26459.0,0.0,0.0,0.0
0,2020-05-09,China,84057.0,0.0,4661.0,0.0,0.0,0.0
0,2020-05-09,Italy,218598.0,0.0,30475.0,0.0,0.0,0.0
0,2020-05-09,Spain,223898.0,0.0,26444.0,0.0,0.0,0.0
0,2020-05-09,United States,1313688.0,0.0,79268.0,0.0,0.0,0.0
0,2020-05-09,World,3999054.0,0.0,280320.0,0.0,0.0,0.0
0,2020-05-09,United Kingdom,217202.0,0.0,31793.0,0.0,0.0,0.0
0,2020-05-09,Germany,169722.0,0.0,7504.0,0.0,0.0,0.0
0,2020-05-09,Iran,106248.0,0.0,6615.0,0.0,0.0,0.0
0,2020-05-09,Turkey,137378.0,0.0,3736.0,0.0,0.0,0.0


In [5]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [6]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [7]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-10,France,139103.0,138854.0,26459.0,26310.0,249.0,149.0
0,2020-05-10,China,84057.0,83991.0,4661.0,4637.0,66.0,24.0
0,2020-05-10,Italy,218598.0,218268.0,30475.0,30395.0,330.0,80.0
0,2020-05-09,Spain,223898.0,223578.0,26444.0,26478.0,320.0,-34.0
0,2020-05-10,United States,1313688.0,1309541.0,79268.0,78794.0,4147.0,474.0
0,2020-05-10,World,3999054.0,3986119.0,280320.0,278814.0,12935.0,1506.0
0,2020-05-10,United Kingdom,217202.0,215260.0,31793.0,31587.0,1942.0,206.0
0,2020-05-10,Germany,169722.0,169218.0,7504.0,7395.0,504.0,109.0
0,2020-05-10,Iran,106248.0,106220.0,6615.0,6589.0,28.0,26.0
0,2020-05-10,Turkey,137378.0,137115.0,3736.0,3739.0,263.0,-3.0


In [8]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")